In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv(r'D:\Big Data\makemytrip.csv',encoding='ISO-8859-1', on_bad_lines='skip')

C:\Users\hp\AppData\Local\Temp\ipykernel_14596\1802023970.py:1: DtypeWarning: Columns (16,34,36,38,39,41,42,43,44,45,48,49,50,51) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'D:\Big Data\makemytrip.csv',encoding='ISO-8859-1', on_bad_lines='skip')


In [3]:
print("Shape :", df.shape)
print("Columns :", df.columns)
print(df.head())

Shape : (20095, 52)
Columns : Index(['area', 'city', 'country', 'crawl_date', 'highlight_value',
       'hotel_overview', 'hotel_ star _rating', 'image_urls', 'in_your_room',
       'is_value_plus', 'latitude', 'longitude', 'mmt_holidayiq_review_count',
       'mmt_location_rating', 'mmt_review_count', 'mmt_review_rating',
       'mmt_review_score', 'mmt_traveller_type_review_count',
       'mmt_tripadvisor_count', 'pageurl', 'property_address', 'property_id',
       'property_name', 'property_type', 'qts', 'query_time_stamp',
       'room_types', 'site_review_count', 'site_review_rating', 'sitename',
       'state', 'traveller_rating', 'uniq_id', 'price', 'Unnamed: 34',
       'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38',
       'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41', 'Unnamed: 42',
       'Unnamed: 43', 'Unnamed: 44', 'Unnamed: 45', 'Unnamed: 46',
       'Unnamed: 47', 'Unnamed: 48', 'Unnamed: 49', 'Unnamed: 50',
       'Unnamed: 51'],
      dtype='object')
       

In [4]:
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [5]:
df.drop_duplicates(inplace=True)

In [6]:
threshold = 0.5 
df = df[df.columns[df.isnull().mean() < threshold]]

In [7]:
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].fillna(df[col].mode()[0])
    else:
        df[col] = df[col].fillna(df[col].median())


In [8]:
for col in df.columns:
    if 'price' in col.lower() or 'rating' in col.lower():
        df[col] = pd.to_numeric(df[col], errors='coerce')

In [9]:
df.fillna(0, inplace=True)

In [10]:
df.to_csv("Hotel_data_cleaned.csv", index=False)

In [11]:
print("Data Cleaned and saved to makemytrip_cleaned.csv")
print("Final Shape :", df.shape)
print("Columns :", df.columns.tolist())

Data Cleaned and saved to makemytrip_cleaned.csv
Final Shape : (20080, 27)
Columns : ['area', 'city', 'country', 'crawl_date', 'highlight_value', 'hotel_overview', 'hotel_ star _rating', 'in_your_room', 'is_value_plus', 'latitude', 'longitude', 'mmt_location_rating', 'mmt_review_rating', 'mmt_traveller_type_review_count', 'mmt_tripadvisor_count', 'pageurl', 'property_address', 'property_id', 'property_name', 'property_type', 'query_time_stamp', 'room_types', 'site_review_count', 'site_review_rating', 'sitename', 'uniq_id', 'price']


In [12]:
from sklearn.preprocessing import MinMaxScaler

In [13]:
df = pd.read_csv("Hotel_data_cleaned.csv", encoding='ISO-8859-1')


In [14]:
print("Available Columns:\n",df.columns)

Available Columns:
 Index(['area', 'city', 'country', 'crawl_date', 'highlight_value',
       'hotel_overview', 'hotel_ star _rating', 'in_your_room',
       'is_value_plus', 'latitude', 'longitude', 'mmt_location_rating',
       'mmt_review_rating', 'mmt_traveller_type_review_count',
       'mmt_tripadvisor_count', 'pageurl', 'property_address', 'property_id',
       'property_name', 'property_type', 'query_time_stamp', 'room_types',
       'site_review_count', 'site_review_rating', 'sitename', 'uniq_id',
       'price'],
      dtype='object')


In [15]:
if 'amenities' in df.columns:
    amenities_dumies =df['amenities'].str.get_dummies(sep=',')
    df = pd.concat([df, amenities_dumies], axis=1)

In [16]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [17]:
for col in df.columns:
    if 'price' in col.lower() or 'rating' in col.lower():
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
        df[[col]] = scaler.fit_transform(df[[col]])

In [18]:
df.to_csv("Hotel_data_features.csv",index=False)
print("Data with features saved to Hotel_data_features.csv")

Data with features saved to Hotel_data_features.csv


In [19]:
from pyspark.sql import SparkSession

In [20]:
spark = SparkSession.builder \
    .appName("HotelRecommendationSystem") \
    .getOrCreate()

In [21]:
df_spark = spark.read.option("header", "true").option("inferSchema", "true").csv("Hotel_data_features.csv")

df_spark.printSchema()
df_spark.show(5)

root
 |-- area: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- crawl_date: string (nullable = true)
 |-- highlight_value: string (nullable = true)
 |-- hotel_overview: string (nullable = true)
 |-- hotel_ star _rating: string (nullable = true)
 |-- in_your_room: string (nullable = true)
 |-- is_value_plus: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- mmt_location_rating: string (nullable = true)
 |-- mmt_review_rating: string (nullable = true)
 |-- mmt_traveller_type_review_count: string (nullable = true)
 |-- mmt_tripadvisor_count: string (nullable = true)
 |-- pageurl: string (nullable = true)
 |-- property_address: string (nullable = true)
 |-- property_id: string (nullable = true)
 |-- property_name: string (nullable = true)
 |-- property_type: string (nullable = true)
 |-- query_time_stamp: string (nullable = true)
 |-- room_types: string (nullable = true)
 |

In [22]:
from pyspark.ml.feature import VectorAssembler
feature_cols = [
    "price",
    "mmt_review_rating",
    "site_review_rating",
    "hotel_star_rating",
    "mmt_location_rating"
]

In [23]:
df_spark = df_spark.withColumnRenamed("hotel_ star _rating", "hotel_star_rating")


In [24]:
for col_name in feature_cols:
    df_spark = df_spark.withColumn(col_name, df_spark[col_name].cast("double"))

assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
df_vectorized = assembler.transform(df_spark)
df_vectorized.select("property_id", "features").show(5, truncate=False)


+-----------+-----------------------------------------------------------------------+
|property_id|features                                                               |
+-----------+-----------------------------------------------------------------------+
|2.01E+17   |[0.24975000000000003,0.0,6.700915791824883E-4,0.01295785413148433,0.0] |
|2.01E+17   |[0.37475,0.0,8.934554389099843E-4,0.03887356239445299,0.0]             |
|2.01E+17   |[0.30074999999999996,0.0,4.4672771945499217E-4,0.02591570826296866,0.0]|
|2.01E+17   |(5,[0,3],[0.911,0.01295785413148433])                                  |
|2.01E+17   |[0.9817500000000001,0.0,4.4672771945499217E-4,0.02591570826296866,0.0] |
+-----------+-----------------------------------------------------------------------+
only showing top 5 rows



In [25]:
from pyspark.ml.feature import Normalizer

normalizer = Normalizer(inputCol="features", outputCol="norm_features", p=2.0)
df_norm = normalizer.transform(df_vectorized)


In [26]:
df_spark.columns
df_spark.select(df_spark.columns).show(5)

+--------------------+-------+-------+----------+--------------------+--------------------+-------------------+-------------------+-------------+---------+---------+-------------------+-----------------+-------------------------------+---------------------+--------------------+--------------------+-----------+--------------------+-------------+--------------------+----------+-----------------+--------------------+----------+--------------------+-------------------+
|                area|   city|country|crawl_date|     highlight_value|      hotel_overview|  hotel_star_rating|       in_your_room|is_value_plus| latitude|longitude|mmt_location_rating|mmt_review_rating|mmt_traveller_type_review_count|mmt_tripadvisor_count|             pageurl|    property_address|property_id|       property_name|property_type|    query_time_stamp|room_types|site_review_count|  site_review_rating|  sitename|             uniq_id|              price|
+--------------------+-------+-------+----------+-----------

In [27]:

drop_cols = ['crawl_date', 'is_value_plus', 'mmt_location_rating', 'mmt_review_rating', 'uniq_id']  
df_spark_cleaned = df_spark.drop(*[col for col in drop_cols if col in df_spark.columns])
numeric_features = ["price", "site_review_rating", "hotel_star_rating"]


In [28]:
for col_name in numeric_features:
    df_spark.select(col_name).summary().show()

+-------+--------------------+
|summary|               price|
+-------+--------------------+
|  count|               19925|
|   mean|6.062735257214603E13|
| stddev| 3.49332373085018E15|
|    min|                 0.0|
|    25%|               0.277|
|    50%| 0.49787500000000007|
|    75%|             0.71575|
|    max|             2.02E17|
+-------+--------------------+

+-------+--------------------+
|summary|  site_review_rating|
+-------+--------------------+
|  count|               19944|
|   mean|3.129261933413573...|
| stddev|2.473248338165631E15|
|    min|                 0.0|
|    25%|2.233638597274960...|
|    50%|4.467277194549921...|
|    75%|6.700915791824883E-4|
|    max|             2.02E17|
+-------+--------------------+

+-------+--------------------+
|summary|   hotel_star_rating|
+-------+--------------------+
|  count|               19883|
|   mean| 0.02383568796737522|
| stddev|0.022932944637389745|
|    min|                 0.0|
|    25%| 0.01295785413148433|
|    5

In [29]:
from pyspark.sql.functions import mean

for col_name in numeric_features:
    mean_val = df_spark.select(mean(col_name)).collect()[0][0]
    df_spark = df_spark.fillna({col_name: mean_val})


In [30]:
from pyspark.sql.functions import mean

for col_name in numeric_features:
    mean_val = df_spark.select(mean(col_name)).collect()[0][0]
    df_spark = df_spark.fillna({col_name: mean_val})

In [31]:
df.shape

(20080, 27)

In [32]:
from pyspark.ml.feature import  MinMaxScaler
numeric_features = [
    "price",
    "site_review_count",
    "site_review_rating",
    "hotel_star_rating",
    "mmt_tripadvisor_count"
]

for col_name in numeric_features:
    df_spark = df_spark.withColumn(col_name, df_spark[col_name].cast("double"))

    df_spark = df_spark.dropna(subset=numeric_features) #for removing rows with null values in numeric features

assembler = VectorAssembler(inputCols=numeric_features, outputCol="numeric_features")
df_temp = assembler.transform(df_spark)

scaler = MinMaxScaler(inputCol="numeric_features", outputCol="scaled_features")
scaler_model = scaler.fit(df_temp)
df_scaled = scaler_model.transform(df_temp)


df_scaled.select("property_id", "scaled_features").show(5, truncate=False)


+-----------+---------------------------------------------------------------------------------------------------------------+
|property_id|scaled_features                                                                                                |
+-----------+---------------------------------------------------------------------------------------------------------------+
|2.01E+17   |[4.119427773178775E-15,4.825870646766169E-16,2.1413726093920012E-17,0.19999999999999998,2.9638822504299583E-17]|
|2.01E+17   |[6.181203435430415E-15,3.383084577114428E-16,2.8551634791893345E-17,0.6,1.682526838904382E-16]                 |
|2.01E+17   |[4.960632243377443E-15,2.4378109452736315E-16,1.4275817395946673E-17,0.39999999999999997,7.914377299934909E-17]|
|2.01E+17   |[1.5026221026489945E-14,3.4328358208955224E-16,0.0,0.19999999999999998,1.4787337355784733E-17]                 |
|2.01E+17   |[1.6193186051324375E-14,5.223880597014925E-16,1.4275817395946673E-17,0.39999999999999997,6.88054664088458

In [33]:
import pyspark.sql.functions as F
from pyspark.sql.functions import udf
from pyspark.ml.linalg import DenseVector
from pyspark.sql.types import FloatType

def extract_feature(vec, idx):
    return float(vec[idx])

extract_udf = udf(extract_feature, FloatType())

for i, col_name in enumerate(numeric_features):
    df_scaled = df_scaled.withColumn(col_name + "_norm", extract_udf("scaled_features", F.lit(i)))


In [34]:
final_features = ["price_norm",
    "site_review_count_norm",
    "site_review_rating_norm",
    "hotel_star_rating_norm",
    "mmt_tripadvisor_count_norm"]  

assembler = VectorAssembler(inputCols=final_features, outputCol="features")
df_vectorized = assembler.transform(df_scaled)

In [35]:
final_features = [
    'price_norm', 'rating_norm', 'location_norm', 'site_review_count_norm',
    'site_review_rating_norm', 'hotel_star_rating_norm', 'mmt_tripadvisor_count_norm'
]

In [36]:
corr_matrix = df.corr(numeric_only=True)
print(corr_matrix["site_review_rating"].sort_values(ascending=False))


site_review_rating     1.000000
longitude              0.010881
site_review_count      0.000187
price                 -0.000219
mmt_location_rating   -0.009625
hotel_ star _rating   -0.009966
mmt_review_rating           NaN
Name: site_review_rating, dtype: float64


In [37]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat_ws, col

In [38]:
spark = SparkSession.builder.appName("HotelRecommender").getOrCreate() #Starting a Spark session
df = spark.read.csv("Hotel_data_features.csv", header=True, inferSchema=True)#Loading data

#Combining text fields
df = df.fillna({'hotel_overview': '', 'highlight_value': '', 'in_your_room': ''})
df = df.withColumn("text_features", concat_ws(" ", "hotel_overview", "highlight_value", "in_your_room"))

In [39]:
from pyspark.sql.functions import when

df = df.withColumn("price", when(col("price") == "N/A", None).otherwise(col("price")).cast("double"))
df = df.withColumn("site_review_rating", when(col("site_review_rating") == "N/A", None).otherwise(col("site_review_rating")).cast("double"))
df = df.withColumn("hotel_star_rating", when(col("hotel_ star _rating") == "N/A", None).otherwise(col("hotel_ star _rating")).cast("double"))


df = df.dropna(subset=["price", "site_review_rating", "hotel_star_rating"]) #dropping the rows with null values


In [40]:
from pyspark.ml.feature import VectorAssembler, MinMaxScaler
from pyspark.sql.functions import col
from pyspark.sql.types import DoubleType

# Converting string columns to double
df = df.withColumn("price", col("price").cast(DoubleType()))
df = df.withColumn("site_review_rating", col("site_review_rating").cast(DoubleType()))
df = df.withColumn("hotel_star_rating", col("hotel_star_rating").cast(DoubleType()))

assembler = VectorAssembler(inputCols=["price", "site_review_rating", "hotel_star_rating"], outputCol="numerical_features")
df = assembler.transform(df)

scaler = MinMaxScaler(inputCol="numerical_features", outputCol="scaled_features")
scaler_model = scaler.fit(df)
df = scaler_model.transform(df)


In [41]:
if "cluster_id" in df.columns:
    df = df.drop("cluster_id")
#just in case of rerunning the program it shows error so this is to resolve this

In [42]:
from pyspark.ml.clustering import KMeans

# Applying K-Means clustering
kmeans = KMeans(featuresCol="scaled_features", predictionCol="cluster_id", k=5, seed=42)
kmeans_model = kmeans.fit(df)
df = kmeans_model.transform(df)

df.select("property_name", "cluster_id").show(5, truncate=False)


+--------------------------------------+----------+
|property_name                         |cluster_id|
+--------------------------------------+----------+
|Zion Home Stay                        |1         |
|Araliayas Resorts                     |1         |
|Shri Udai Palace                      |1         |
|SNP House Airport Hotel And Restaurant|3         |
|Hotel Pichola Haveli (LAKE SIDE)      |3         |
+--------------------------------------+----------+
only showing top 5 rows



In [43]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF

tokenizer = Tokenizer(inputCol="text_features", outputCol="words")
df = tokenizer.transform(df)

remover = StopWordsRemover(inputCol="words", outputCol="filtered")
df = remover.transform(df)

hashingTF = HashingTF(inputCol="filtered", outputCol="raw_features", numFeatures=10000)
df = hashingTF.transform(df)

idf = IDF(inputCol="raw_features", outputCol="tfidf_features")
idf_model = idf.fit(df)
df = idf_model.transform(df)


In [44]:
def recommend_similar_hotels(spark_df, hotel_name, top_n=5):
    hotel_row = spark_df.filter(col("property_name") == hotel_name).collect()[0]
    cluster_id = hotel_row["cluster_id"]

    cluster_df = spark_df.filter(col("cluster_id") == cluster_id).select("property_name", "tfidf_features")

    pandas_df = cluster_df.toPandas()

    from sklearn.metrics.pairwise import cosine_similarity
    import numpy as np

    tfidf_matrix = np.vstack(pandas_df["tfidf_features"].values)
    similarities = cosine_similarity([hotel_row["tfidf_features"]], tfidf_matrix)[0]

    pandas_df["similarity"] = similarities
    results = pandas_df[pandas_df["property_name"] != hotel_name].sort_values("similarity", ascending=False).head(top_n)

    return results[["property_name", "similarity"]]


In [45]:
recommendations = recommend_similar_hotels(df, "SNP House Airport Hotel And Restaurant", top_n=5)
print(recommendations)
#just to check if the recommendations are working

                        property_name  similarity
1    Hotel Pichola Haveli (LAKE SIDE)    0.285795
390              Hotel Krishna Palace    0.212195
10            Janak Niwas Guest House    0.205688
950                    Alsisar Haveli    0.196754
385           Janak Niwas Guest House    0.185316


In [46]:
def recommend_filtered_hotels(spark_df, hotel_name, city=None, min_price=None, max_price=None, amenity_keywords=None, top_n=5):
    hotel_row = spark_df.filter(col("property_name") == hotel_name).collect()[0]
    cluster_id = hotel_row["cluster_id"]

    filtered_df = spark_df.filter(col("cluster_id") == cluster_id)

    if city:
        filtered_df = filtered_df.filter(col("city") == city)
    if min_price is not None:
        filtered_df = filtered_df.filter(col("price") >= min_price)
    if max_price is not None:
        filtered_df = filtered_df.filter(col("price") <= max_price)
    if amenity_keywords:
        for keyword in amenity_keywords:
            filtered_df = filtered_df.filter(col("in_your_room").contains(keyword))

    pandas_df = filtered_df.select("property_name", "tfidf_features").toPandas()

    from sklearn.metrics.pairwise import cosine_similarity
    import numpy as np

    if pandas_df.empty:
        return "No results match the filters."

    tfidf_matrix = np.vstack(pandas_df["tfidf_features"].values)
    similarities = cosine_similarity([hotel_row["tfidf_features"]], tfidf_matrix)[0]

    pandas_df["similarity"] = similarities
    pandas_df = pandas_df[pandas_df["property_name"] != hotel_name]
    return pandas_df.sort_values("similarity", ascending=False).head(top_n)[["property_name", "similarity"]]


In [47]:
recommendations = recommend_similar_hotels(df, "Hotel Pichola Haveli (LAKE SIDE)", top_n=5)
print(recommendations)
#just to check if the recommendations are working

                               property_name  similarity
2286  SNP House Airport Hotel And Restaurant    0.303638
10                   Janak Niwas Guest House    0.290962
0     SNP House Airport Hotel And Restaurant    0.285795
2796                        Hotel Sun Palace    0.280913
425                     Hotel Jasol Heritage    0.270198


In [51]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np

# Step 1: Load your hotel data
df = pd.read_csv("Hotel_data_cleaned.csv")

# Step 2: Fill missing text with empty string (important for vectorizer)
df["in_your_room"] = df["in_your_room"].fillna("")

# Step 3: Apply TF-IDF Vectorizer
vectorizer = TfidfVectorizer(stop_words="english")
tfidf_matrix = vectorizer.fit_transform(df["in_your_room"])

# Step 4: Convert sparse matrix to dense feature array
tfidf_features = tfidf_matrix.toarray()

# Step 5: Store vectors as list in a new column
df["tfidf_features"] = tfidf_features.tolist()

# Step 6: Cluster using KMeans (you can choose n_clusters = 5 or more)
kmeans = KMeans(n_clusters=5, random_state=42)
df["cluster_id"] = kmeans.fit_predict(tfidf_features)

# Step 7: Save the final data
df.to_csv("Hotel_data_with_clusters.csv", index=False)

print("✅ tfidf_features and cluster_id generated successfully.")


✅ tfidf_features and cluster_id generated successfully.
